In [1]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 77840, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 77840 (delta 129), reused 164 (delta 100), pack-reused 77621
Receiving objects: 100% (77840/77840), 593.42 MiB | 48.71 MiB/s, done.
Resolving deltas: 100% (55305/55305), done.


In [2]:
import os
os.environ['PYTHONPATH'] += ":/content/models:/content/models/research/:/content/models/research/slim/"

import sys
sys.path.append("/content/models:/content/models:/content/models/research/:/content/models/research/slim/")

In [3]:
!pip3 install tf-models-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 5.2 MB/s 
     |████████████████████████████████| 238 kB 78.7 MB/s 
     |████████████████████████████████| 352 kB 83.7 MB/s 
     |████████████████████████████████| 5.9 MB 74.1 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 578.0 MB 12 kB/s 
     |████████████████████████████████| 116 kB 86.1 MB/s 
     |████████████████████████████████| 636 kB 77.4 MB/s 
     |████████████████████████████████| 1.1 MB 81.0 MB/s 
     |████████████████████████████████| 38.2 MB 68.5 MB/s 
     |████████████████████████████████| 99 kB 9.4 MB/s 
     |████████████████████████████████| 1.3 MB 72.7 MB/s 
     |████████████████████████████████| 1.7 MB 71.4 MB/s 
     |████████████████████████████████| 438 kB 84.4 MB/s 
     |████████████████████████████████| 5.9 MB 71.7 MB/s 
  Created wheel for py-cpuinfo: fil

In [4]:
!pip3 install bert-for-tf2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 141 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=0ee7f70839068c7c00ec735d6fb887423ade435316f6752e2d4beaab462aad17
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=b6bbd0ddd3f5ec04a434689b30365217726d53921c88662cc41c92137026c56e
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=13bc1459aad866edbbb20b5258a564fb4b076a0977b009fe99cd2d87df3fac8e
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow p

In [5]:
import functools
import math
import collections
import csv
import os

from absl import app
from absl import flags
from absl import logging
import tensorflow as tf

Import Tensorflow BERT and NLP packages

In [6]:
from official.common import distribute_utils
from official.nlp import optimization
from official.nlp.modeling import networks
from official.utils.misc import keras_utils
import warnings
warnings.filterwarnings("ignore")

Authentication for accessing Google Cloud storage bucket

In [7]:
from google.colab import auth
auth.authenticate_user()

In [8]:
CLOUD_PROJECT = 'strategic-cacao-364520'
BUCKET = 'gs://cds-sbert'

In [9]:
!gcloud config set project $CLOUD_PROJECT

Updated property [core/project].


In [10]:
bert_train_file = BUCKET +  "/input-data/bctrain.txt"
bert_test_file = BUCKET +  "/input-data/bcdev.txt"
model_dir = BUCKET + '/bert_model9'

Initializing SBERT Model Parameters

In [11]:
train_data_path = bert_train_file
eval_data_path = None
init_checkpoint = None

max_seq_length = 128

train_batch_size = 32
train_data_size = 363846
eval_data_size = 10000
num_train_epochs = 3
eval_batch_size = 16

pooling_mech = 'mean'                        # ['cls', 'mean', 'max']
compute_similarity = 'dense'                 # ['cosine_similarity', 'dense']
cmode = 'train_and_eval'                     # ['train_and_eval', 'predict']
learning_rate = 3e-5
vocab_size = 30522

distribution_strategy = 'tpu'
tpu = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', tpu)

TPU address is grpc://10.112.193.2:8470


Functions to convert Questions text to Ids (tokens, segment and masks)

In [12]:
def sentence_to_ids(sentence, tokenizer, max_seq_length):
    # Consider [CLS] and [SEP]
    tokenized = tokenizer.tokenize(sentence)
    tokens = ["[CLS]"]
    tokens.extend(tokenized[0:(max_seq_length - 2)])
    tokens.append("[SEP]")

    ids = tokenizer.convert_tokens_to_ids(tokens)
    mask = [1] * len(ids)
    seg_id = 0
    segment_ids = [seg_id] * max_seq_length
  
    while len(ids) < max_seq_length:
        ids.append(0)
        mask.append(0)

    assert len(ids) == max_seq_length
    assert len(mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    def create_feature(values):
        f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
        return f

    return create_feature(ids), create_feature(mask), create_feature(segment_ids)

def convert_single_example(features, tokenizer, max_seq_length):
    """Converts a single `InputExample` into a single `InputFeatures`."""
    assert len(features) == 6
    idx = int(features[0])
    sentence0 = features[3]
    sentence1 = features[4]
    label = float(features[5])

    features = collections.OrderedDict()
    features["id"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[idx]))
    (features["left_input_ids"], features["left_input_mask"],
        features["left_segment_ids"]) = sentence_to_ids(sentence0, tokenizer, max_seq_length)
    (features["right_input_ids"], features["right_input_mask"],
        features["right_segment_ids"]) = sentence_to_ids(sentence1, tokenizer, max_seq_length)
    features["label"] = tf.train.Feature(float_list=tf.train.FloatList(value=[label]))

    return tf.train.Example(features=tf.train.Features(feature=features)), label == 1.0

Function to prepare inputs to the SBERT Classifier

In [13]:
def create_dataset(bert_input_file, seq_length, batch_size, is_training=True, input_pipeline_context=None):
    """Creates a TF dataset."""
    dataset = tf.data.TFRecordDataset(bert_input_file)

    features = {
      'left_input_ids': tf.io.FixedLenFeature([seq_length], tf.int64),
      'left_input_mask': tf.io.FixedLenFeature([seq_length], tf.int64),
      'left_segment_ids': tf.io.FixedLenFeature([seq_length], tf.int64),
      'right_input_ids': tf.io.FixedLenFeature([seq_length], tf.int64),
      'right_input_mask': tf.io.FixedLenFeature([seq_length], tf.int64),
      'right_segment_ids': tf.io.FixedLenFeature([seq_length], tf.int64),
      'label': tf.io.FixedLenFeature([], tf.float32),
    }

    dataset = dataset.map( lambda record: tf.io.parse_single_example(record, features), num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if input_pipeline_context and input_pipeline_context.num_input_pipelines > 1:
        dataset = dataset.shard(input_pipeline_context.num_input_pipelines, input_pipeline_context.input_pipeline_id)

    def _select_data_from_record(record):
        x = {
            'left_input_ids': record['left_input_ids'],
            'left_input_mask': record['left_input_mask'],
            'left_input_type_ids': record['left_segment_ids'],
            'right_input_ids': record['right_input_ids'],
            'right_input_mask': record['right_input_mask'],
            'right_input_type_ids': record['right_segment_ids']
        }
        y = record['label']
        return (x, y)

    if is_training:
        dataset = dataset.shuffle(100)
        dataset = dataset.repeat()

    dataset = dataset.map(_select_data_from_record, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size, drop_remainder=is_training)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


SBERT Classifier

In [14]:
class SBERT_Classifier(tf.keras.Model):
    """Sequence similarity classifier.
        Args:
        encoder: a tf.keras.Model defines an encoder network.
        max_seq_length: maximum sequence length.
        poolining: poolining mechanism. One of [cls, mean, max].
        compute_similarity: how to compute the similarity probability outputs. One of [cosine_similarity, dense].
    """

    def __init__(self, encoder, max_seq_length, dropout_rate=0.1, pooling='mean', compute_similarity='dense', **kwargs):
        self.max_seq_length = max_seq_length
        self.pooling = pooling
        self.compute_similarity = compute_similarity
        
        # Prepare inputs
        left_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='left_word_ids')
        left_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='left_mask')
        left_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='left_type_ids')
        left_inputs = [left_word_ids, left_mask, left_type_ids]

        right_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='right_word_ids')
        right_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='right_mask')
        right_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='right_type_ids')
  
        # Normally the question word is the second token.
        left_q_word = tf.slice(left_word_ids, [0, 1], [-1, 1])
        right_q_word = tf.slice(right_word_ids, [0, 1], [-1, 1])
        question_word_diff = tf.cast(tf.not_equal(left_q_word, right_q_word), tf.float32)

        right_inputs = [right_word_ids, right_mask, right_type_ids]
        inputs = {
            'left_input_ids': left_word_ids,
            'left_input_mask': left_mask,
            'left_input_type_ids': left_type_ids,
            'right_input_ids': right_word_ids,
            'right_input_mask': right_mask,
            'right_input_type_ids': right_type_ids
        }

        expanded_left_mask = tf.expand_dims(tf.cast(left_mask, dtype=tf.float32), axis=2)
        expanded_right_mask = tf.expand_dims(tf.cast(right_mask, dtype=tf.float32), axis=2)
      
        # Get sequence embeddings
        left_sequence_output, left_cls_output = encoder(left_inputs)
        right_sequence_output, right_cls_output = encoder(right_inputs)
        left_sequence_output = left_sequence_output * expanded_left_mask
        right_sequence_output = right_sequence_output * expanded_right_mask

        # Pooling
        if pooling == 'cls':
            # Outputs on the first token 'CLs'
            left_outputs = left_cls_output
            right_outputs = right_cls_output
        elif pooling == 'mean':
            left_outputs = tf.reduce_sum(left_sequence_output, axis=1) / tf.reduce_sum(expanded_left_mask, axis=1)
            right_outputs = tf.reduce_sum(right_sequence_output, axis=1) / tf.reduce_sum(expanded_right_mask, axis=1)
        elif pooling == 'max':
            left_outputs = tf.reduce_max(left_sequence_output, axis=1)
            right_outputs = tf.reduce_max(right_sequence_output, axis=1)
        else:
            raise ValueError('Pooling %s is not supported: %s' % pooling)

        # Compute similarity
        if compute_similarity == 'cosine_similarity':
            cos_similarity = tf.reduce_sum(tf.nn.l2_normalize(left_outputs, axis=1) * tf.nn.l2_normalize(right_outputs, axis=1), axis=1)
            prob = (cos_similarity + 1) / 2
        elif compute_similarity == 'dense':
            concat_outputs = tf.concat([left_outputs, right_outputs, tf.abs(left_outputs - right_outputs)], axis=1)
            outputs = tf.keras.layers.Dense(128, activation='relu')(concat_outputs)
            # skip connection on question words
            outputs = tf.concat([outputs, question_word_diff], axis=1)
            prob = tf.keras.layers.Dense(1, activation='sigmoid')(outputs)
        else:
            raise ValueError('compute_similarity %s is not supported: %s' %compute_similarity)
   
        super(SBERT_Classifier, self).__init__(inputs=inputs, outputs=prob, **kwargs)
        self._encoder = encoder

        config_dict = {
            'encoder': self._encoder,
            'max_seq_length': self.max_seq_length,
            'pooling': self.pooling,
            'compute_similarity': self.compute_similarity
        }
        config_cls = collections.namedtuple('Config', config_dict.keys())
        self._config = config_cls(**config_dict)

    @property
    def checkpoint_items(self):
        items = dict(encoder=self._encoder)
        return items

    def get_config(self):
         return dict(self._config._asdict())
        
    @classmethod
    def from_config(cls, config, custom_objects=None):
        return cls(**config)

Functions to run SBERT Classifier, Trian and Evaluate, Predict

In [15]:
from typing import Any
def get_loss_fn():
    """Gets the binary classification loss function."""
    def classification_loss_fn(labels, logits):
        binary_cross_entropy = tf.keras.losses.BinaryCrossentropy(
            label_smoothing=0.1, reduction=tf.keras.losses.Reduction.SUM)
        return binary_cross_entropy(labels, logits)
    return classification_loss_fn

def get_dataset_fn(input_file, max_seq_length, global_batch_size, is_training):
    """Gets a closure to create a dataset."""
    def _dataset_fn(ctx=None):
        batch_size = ctx.get_per_replica_batch_size(global_batch_size) if ctx else global_batch_size
        ds = create_dataset(input_file, max_seq_length, batch_size, is_training=is_training, input_pipeline_context=ctx)
        return ds
    return _dataset_fn

def run_bertclassifier(strategy, model_dir, epochs, steps_per_epoch, eval_steps,
                            warmup_steps, init_checkpoint, train_input_fn, eval_input_fn):
    """Train the SBERT classifier."""
  
    pooling = pooling_mech

    loss_fn = (get_loss_fn())
   
    # Start training using Keras compile/fit API.
    with strategy.scope():
        training_dataset = train_input_fn()
        evaluation_dataset = eval_input_fn() if eval_input_fn else None
        enc = networks.BertEncoder(vocab_size=vocab_size, num_layers=12, type_vocab_size=2)
        classifier = SBERT_Classifier(enc, max_seq_length, pooling=pooling, compute_similarity=compute_similarity)
        classifier.optimizer = optimization.create_optimizer(learning_rate, steps_per_epoch * epochs, warmup_steps)
        optimizer = classifier.optimizer

        if init_checkpoint:
            # load pretrained weights
            classifier.load_weights(init_checkpoint).assert_existing_objects_matched()
        
        loss_weights = None
        metrics_list = [tf.metrics.BinaryAccuracy(), tf.metrics.Precision(), tf.metrics.Recall()]

        classifier.compile(
            optimizer=optimizer,
            loss=loss_fn,
            loss_weights=loss_weights,
            metrics=metrics_list, 
            steps_per_execution=100)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=model_dir, save_weights_only=True,  verbose=1)
        
        callbacks = [cp_callback]

        history = classifier.fit(
            x=training_dataset,
            validation_data=evaluation_dataset,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            validation_steps=eval_steps,
            callbacks=callbacks)
        stats = {'total_training_steps': steps_per_epoch * epochs}
        if 'loss' in history.history:
            stats['train_loss'] = history.history['loss'][-1]
        if 'val_accuracy' in history.history:
            stats['eval_metrics'] = history.history['val_accuracy'][-1]
        return classifier, stats

def train_and_eval(strategy, train_input_fn=None, eval_input_fn=None, init_checkpoint=None):
    """Run training and evaluation."""
    epochs = num_train_epochs
    steps_per_epoch = int(train_data_size // train_batch_size)
    warmup_steps = int(epochs * train_data_size * 0.1 // train_batch_size)
    eval_steps = int(math.ceil(eval_data_size // eval_batch_size))

    trained_model, _ = run_bertclassifier(strategy, model_dir, epochs, steps_per_epoch, eval_steps, warmup_steps,
                                      init_checkpoint, train_input_fn, eval_input_fn)

    return trained_model

def get_predictions_and_labels(strategy, trained_model, eval_input_fn):
    """Obtains predictions of trained model on test data."""

    @tf.function
    def predict_step(iterator):
        """Computes predictions on distributed devices."""

        def _predict_step_fn(inputs):
            """Replicated predictions."""
            inputs, labels = inputs
            probabilities = trained_model(inputs, training=False)
            return probabilities, labels

        outputs, labels = strategy.run(_predict_step_fn, args=(next(iterator),))
        outputs = tf.nest.map_structure(strategy.experimental_local_results, outputs)
        labels = tf.nest.map_structure(strategy.experimental_local_results, labels)
        return outputs, labels

    test_iter = iter(strategy.distribute_datasets_from_function(eval_input_fn))
    all_predictions, all_labels = list(), list()
    try:
        with tf.experimental.async_scope():
            while True:
                probs, labels = predict_step(test_iter)
                for prob, label in zip(probs, labels):
                    all_predictions.extend(prob.numpy())
                    all_labels.extend(label.numpy())
    except (StopIteration, tf.errors.OutOfRangeError):
        tf.experimental.async_clear_error()

    return all_predictions, all_labels


Request cloud TPU environment to run same Tensorflow version as current colab notebook

In [16]:
import requests
import os
url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/2.10.0'
resp = requests.post(url)
print(resp)

<Response [200]>


Set TPU distribution strategy to train Sbert model

In [17]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.112.193.2:8470']
REPLICAS:  8


In [19]:
"""Train the classification Model."""
eval_input_fn = None 
#init_checkpoint = None
init_checkpoint = model_dir
train_input_fn = get_dataset_fn(train_data_path, max_seq_length, train_batch_size, is_training=True)
bertModel = train_and_eval(strategy, train_input_fn, eval_input_fn, init_checkpoint)

Epoch 1/3


11370/11370 [==============================] - ETA: 0s - loss: 0.9670 - binary_accuracy: 0.9777 - precision_1: 0.9603 - recall_1: 0.9801
Epoch 1: saving model to gs://cds-sbert/bert_model9
11370/11370 [==============================] - 797s 70ms/step - loss: 0.9670 - binary_accuracy: 0.9777 - precision_1: 0.9603 - recall_1: 0.9801
Epoch 2/3
11370/11370 [==============================] - ETA: 0s - loss: 0.9680 - binary_accuracy: 0.9776 - precision_1: 0.9602 - recall_1: 0.9801
Epoch 2: saving model to gs://cds-sbert/bert_model9
11370/11370 [==============================] - 728s 64ms/step - loss: 0.9680 - binary_accuracy: 0.9776 - precision_1: 0.9602 - recall_1: 0.9801
Epoch 3/3
11370/11370 [==============================] - ETA: 0s - loss: 0.9688 - binary_accuracy: 0.9774 - precision_1: 0.9601 - recall_1: 0.9795
Epoch 3: saving model to gs://cds-sbert/bert_model9
11370/11370 [==============================] - 729s 64ms/step - loss: 0.9688 - binary_accuracy: 0.9774 - precision_1: 0.9601 

Predict on Test data

In [20]:
test_input_fn = get_dataset_fn(bert_test_file, max_seq_length, train_batch_size, is_training=False)
preds, labels = get_predictions_and_labels(strategy, bertModel, test_input_fn)

In [44]:
# probability threshold of 70% for positive duplicate
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
import numpy as np
ypred = np.array(preds).reshape(-1)
rundoff = lambda x: 1 if x > 0.7 else 0
vfunc = np.vectorize(rundoff)
ypreds = vfunc(ypred)
#prediction1 = np.round(abs(ypred))
print ('Accuracy:  ', accuracy_score(labels, ypreds))
print ('F1 score:  ', f1_score(labels, ypreds))
print ('Recall:    ', recall_score(labels, ypreds))
print ('Precision: ', precision_score(labels, ypreds))
print ('\nclasification report:\n\n', classification_report(labels,ypreds))
print ('\n confusion matrix:\n\n',confusion_matrix(labels, ypreds))

Accuracy:   0.8557754142963147
F1 score:   0.8100342075256557
Recall:     0.8352032247228753
Precision:  0.786337760910816

clasification report:

               precision    recall  f1-score   support

         0.0       0.90      0.87      0.88     25545
         1.0       0.79      0.84      0.81     14885

    accuracy                           0.86     40430
   macro avg       0.84      0.85      0.85     40430
weighted avg       0.86      0.86      0.86     40430


 confusion matrix:

 [[22167  3378]
 [ 2453 12432]]


In [41]:
# probability threshold of 75% for positive duplicate

ypred = np.array(preds).reshape(-1)
rundoff = lambda x: 1 if x > 0.75 else 0
vfunc = np.vectorize(rundoff)
ypreds = vfunc(ypred)
#prediction1 = np.round(abs(ypred))
print ('Accuracy:  ', accuracy_score(labels, ypreds))
print ('F1 score:  ', f1_score(labels, ypreds))
print ('Recall:    ', recall_score(labels, ypreds))
print ('Precision: ', precision_score(labels, ypreds))
print ('\nclasification report:\n\n', classification_report(labels,ypreds))
print ('\n confusion matrix:\n\n',confusion_matrix(labels, ypreds))

Accuracy:   0.8569131832797428
F1 score:   0.8103713901727472
Recall:     0.8304333221363789
Precision:  0.7912559211368583

clasification report:

               precision    recall  f1-score   support

         0.0       0.90      0.87      0.89     25545
         1.0       0.79      0.83      0.81     14885

    accuracy                           0.86     40430
   macro avg       0.84      0.85      0.85     40430
weighted avg       0.86      0.86      0.86     40430


 confusion matrix:

 [[22284  3261]
 [ 2524 12361]]


In [45]:
# probability threshold of 80% for positive duplicate

ypred = np.array(preds).reshape(-1)
rundoff = lambda x: 1 if x > 0.8 else 0
vfunc = np.vectorize(rundoff)
ypreds = vfunc(ypred)
#prediction1 = np.round(abs(ypred))
print ('Accuracy:  ', accuracy_score(labels, ypreds))
print ('F1 score:  ', f1_score(labels, ypreds))
print ('Recall:    ', recall_score(labels, ypreds))
print ('Precision: ', precision_score(labels, ypreds))
print ('\nclasification report:\n\n', classification_report(labels,ypreds))
print ('\n confusion matrix:\n\n',confusion_matrix(labels, ypreds))

Accuracy:   0.857803611179817
F1 score:   0.8102327116685921
Recall:     0.8245213301981861
Precision:  0.7964308890330954

clasification report:

               precision    recall  f1-score   support

         0.0       0.90      0.88      0.89     25545
         1.0       0.80      0.82      0.81     14885

    accuracy                           0.86     40430
   macro avg       0.85      0.85      0.85     40430
weighted avg       0.86      0.86      0.86     40430


 confusion matrix:

 [[22408  3137]
 [ 2612 12273]]


In [42]:
import pandas as pd
list_of_tuples = list(zip(preds, ypreds,labels))
tst = pd.DataFrame(list_of_tuples, columns=['prob','pred', 'label'])

tst.to_csv('sbert_predicted_75_output.csv')

Misc - backup for creating Word tags, Segments and Masks for train data

In [ ]:
with strategy.scope():
    enc = networks.BertEncoder(vocab_size=vocab_size, num_layers=12, type_vocab_size=2)
    base_classifier = SBERT_Classifier(enc, max_seq_length, pooling=pooling_mech, compute_similarity=compute_similarity,
                                    question_word_penalty_weight=question_word_penalty_weight, pred_threshold=pred_threshold)
classifier = base_classifier
checkpoint = tf.train.Checkpoint(model=classifier)
latest_checkpoint_file = (init_checkpoint or tf.train.latest_checkpoint(model_dir))
assert latest_checkpoint_file
logging.info('Checkpoint file %s found and restoring from '
            'checkpoint', latest_checkpoint_file)
checkpoint.restore(latest_checkpoint_file).assert_existing_objects_matched()
preds, labels = get_predictions_and_labels(strategy, classifier, eval_input_fn)
#write_prediction_outputs(preds, labels)


Backup to create Train and Test data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train1_data = pd.read_csv(input_file, delimiter=",", encoding='utf-8')
train_data = train1_data.sample(frac=1)
train=train1_data.sample(frac=0.8,random_state=42) #random state is a seed value
test=train1_data.drop(train.index)

train.to_csv(r'/content/drive/MyDrive/Copy-CDS-B3/Capstone Project/Data/cleantext/bert_train_data.csv', index=None, sep=',', mode='a')
test.to_csv(r'/content/drive/MyDrive/Copy-CDS-B3/Capstone Project/Data/cleantext/bert_test_data.csv', index=None, sep=',', mode='a')

In [ ]:
input_file = "/content/drive/MyDrive/Copy-CDS-B3/Capstone Project/Data/cleantext/bert/dev.tsv"
output_file = "/content/drive/MyDrive/Copy-CDS-B3/Capstone Project/Data/cleantext/bert/bdev.txt"
vocab_file = "/content/drive/MyDrive/Copy-CDS-B3/Capstone Project/Data/cleantext/bert/vocab.txt"

In [ ]:
writer = tf.io.TFRecordWriter(output_file)

from bert import bert_tokenization

example_idx = 0
logging.info("Start.")
pos_num = 0
  
# WordPiece tokenization
tokenizer = bert_tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=True)

with tf.io.gfile.GFile(input_file, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    next(reader)
    examples = []
    for features in reader:
        tf_example, pos = convert_single_example(features, tokenizer, max_seq_length)
        if pos:
            pos_num = pos_num + 1
        writer.write(tf_example.SerializeToString())

        example_idx = example_idx + 1
        if example_idx % 10000 == 0:
            logging.info("Finished writing %d examples", example_idx)
        logging.info("Finished writing %d examples", example_idx)
        logging.info("Positive %d examples", pos_num)
writer.close()